# Introduction to Encrypted Tensors

Following along here: <https://www.youtube.com/watch?v=CLunSEdSDaA>

## Docs

Main: <https://crypten.readthedocs.io/en/latest/mpctensor.html>

- CrypTensor: https://crypten.readthedocs.io/en/latest/cryptensor.html
- MPCTensor: https://crypten.readthedocs.io/en/latest/mpctensor.html
- Neural Nets: https://crypten.readthedocs.io/en/latest/nn.html

In [ ]:
import sys

import torch
import torchvision
import crypten

assert sys.version_info[0] == 3 and sys.version_info[1] == 7, "python 3.7 is required!"

print(f"Okay, good! You have: {sys.version_info[:3]}")
# Now we can init crypten!
crypten.init()

In [ ]:
x = crypten.cryptensor([1, 2, 3])
x

In [ ]:
# Make it readable
x.get_plain_text()

## Let's test some operations

More operations here: [docs](https://crypten.readthedocs.io/en/latest/cryptensor.html#tensor-operations)

In [ ]:
a = (2+x)
a.get_plain_text()

In [ ]:
b = (a+x)
b.get_plain_text()

In [ ]:
c = x*a
c.get_plain_text()

In [ ]:
d = x.dot(a)
d.get_plain_text()

In [ ]:
# Lets compute Mean Squared Loss

sql = (x - c)**2
msql = sql.mean()

msql.get_plain_text()

In [ ]:
# The pytorch version
x_pt = torch.tensor([1,2,3.])
c_pt = x_pt*(2+x_pt)

sql_pt = (x_pt - c_pt)**2
msql_pt = sql_pt.mean().abs()
print(msql_pt)

## Neural Nets

[Docs](https://crypten.readthedocs.io/en/latest/nn.html)

`crypten.nn` provides modules for defining and training neural networks similar to `torch.nn`.

### From PyTorch to CrypTen

The simplest way to create a CrypTen network is to start with a PyTorch network, and use the `from_pytorch` function to convert it to a CrypTen network. This is particularly useful for pre-trained PyTorch networks that need to be encrypted before use.

In [ ]:
# Load parent folders into path
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
# Import some config variables
from config import PETER_ROOT, DATA_DIR, MNIST_SIZE

# Load a pytorch net
from ZeNet.nets import *

# Plotting
from plot_mnist import plot_batch

In [ ]:
torch.set_num_threads(1)

subset = 1/60
train_ratio = 0.75
test_ratio = 1 - train_ratio
batch_size_train = int((subset * MNIST_SIZE) * train_ratio)
batch_size_test = int((subset * MNIST_SIZE) * test_ratio)

print(f"Using train_test ratios: {train_ratio} : {test_ratio}")
print(f"Train batch size: {batch_size_train}")
print(f"Test batch size: {batch_size_test}")

In [ ]:
net = Net1()
net

In [ ]:
# Load data if needed
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST(DATA_DIR, train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST(DATA_DIR, train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

  # If he needs to download it, cause it's not already in the data folder, he/she/it would do so and say so below.

In [ ]:
examples = enumerate(test_loader)
batch_idx, (data, targets) = next(examples)

In [ ]:
batch_idx, data.shape, targets.shape

In [ ]:
print(f"type: {type(data)}")
print(data.shape)

print(f"Means that we have {data.shape[0]} images of size {data.shape[2]}x{data.shape[3]} in {data.shape[1]} color channels (1 channel = greyscale)")

In [ ]:
plot_batch(data, targets)

## Private sharing of samples and labels

We're going to stick to CrypTens ABC naming "convention", so

- Alice = 0
- Bob = 1
- Carl = 2
- Daniel = 3
- ...

## Setup 1: Alice has samples, Bob has lables

Maybe this doesn't seem to applicable, but think of something like healthcare data instead:

- Alice has general data about a person
- Bob has diagnostic data about the patient

We'd like to learn how to predict the likelyhood of someone falling ill to a specific illness, but don't want to get in trouble with the EU over Data Privacy laws!

### Assign ranks to each participant

In [ ]:
from pathlib import Path

In [ ]:

ALICE = 0
BOB = 1
CARL = 2

participants = ["alice", "bob"]
TMP_DIR = Path("./TMP")
print(f"Our temporary data will land here: {TMP_DIR}")

In [ ]:
from crypten import mpc

# Specify file locations to save each piece of data
filenames = {
    "features": "/tmp/features.pth",
    "labels": "/tmp/labels.pth",
    "b_true": "/tmp/b_true.pth",
    "test_features": "/tmp/test_features.pth",
    "test_labels": "/tmp/test_labels.pth",
    "w_true": "/tmp/w_true.pth",
}

for u in participants:
    filenames["features_"+u] = TMP_DIR / ("feautures_" + u)
    filenames["samples_"+u] = TMP_DIR / ("samples_" + u)



In [ ]:

@mpc.run_multiprocess(world_size=2)
def save_all_data():
    # Save features, labels for Data Labeling example
    crypten.save(features, filenames["features"])
    crypten.save(labels, filenames["labels"])
    
    # Save split features for Feature Aggregation example
    features_alice = features[:50]
    features_bob = features[50:]
    
    crypten.save(features_alice, filenames["features_alice"], src=ALICE)
    crypten.save(features_bob, filenames["features_bob"], src=BOB)
    
    # Save split dataset for Dataset Aggregation example
    samples_alice = features[:, :500]
    samples_bob = features[:, 500:]
    crypten.save(samples_alice, filenames["samples_alice"], src=ALICE)
    crypten.save(samples_bob, filenames["samples_bob"], src=BOB)
    
    # Save true model weights and biases for Model Hiding example
    crypten.save(w_true, filenames["w_true"], src=ALICE)
    crypten.save(b_true, filenames["b_true"], src=ALICE)
    
    crypten.save(test_features, filenames["test_features"], src=BOB)
    crypten.save(test_labels, filenames["test_labels"], src=BOB)
    
save_all_data()